In [1]:
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
import tensorflow as tf

mnist = datasets.fetch_mldata('MNIST original', data_home='.')

Using TensorFlow backend.
D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
n = len(mnist.data)
N = 10000
indices = np.random.permutation(range(n))[:N]
X = mnist.data[indices]
y = mnist.target[indices]
Y = np.eye(10)[y.astype(int)]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

In [4]:
n_in = len(X[0]) # 784
n_hidden = 200
n_out = len(Y[0]) # 10

x = tf.placeholder(tf.float32, shape=[None, n_in])
t = tf.placeholder(tf.float32, shape=[None, n_out])
keep_prob = tf.placeholder(tf.float32)

# 입력층 - 은닉층
W0 = tf.Variable(tf.truncated_normal([n_in, n_hidden], stddev=0.01))
b0 = tf.Variable(tf.zeros([n_hidden]))
h0 = tf.nn.relu(tf.matmul(x, W0) + b0)
h0_drop = tf.nn.dropout(h0, keep_prob)

# 은닉층 - 은닉층
W1 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b1 = tf.Variable(tf.zeros([n_hidden]))
h1 = tf.nn.relu(tf.matmul(h0_drop, W1) + b1)
h1_drop = tf.nn.dropout(h1, keep_prob)

W2 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b2 = tf.Variable(tf.zeros([n_hidden]))
h2 = tf.nn.relu(tf.matmul(h1_drop, W2) + b2)
h2_drop = tf.nn.dropout(h2, keep_prob)

# 은닉층 - 출력층
W3 = tf.Variable(tf.truncated_normal([n_hidden, n_out], stddev=0.01))
b3 = tf.Variable(tf.zeros([n_out]))
y = tf.nn.softmax(tf.matmul(h2_drop, W3) + b3)

In [5]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [7]:
cross_entropy = -tf.reduce_sum(t * tf.log(y) + (1-t)*tf.log(y-1))
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
epochs = 1000
batch_size = 20
n_batches = 100

for epoch in range(epochs):
    X_, Y_ = shuffle(X_train, Y_train)
    
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        
        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end],
            keep_prob: 0.5
        })    

In [15]:
accuracy_rate = accuracy.eval(session=sess, feed_dict={
    x: X_test,
    t: Y_test,
    keep_prob: 1.0
})
print('accuracy: ', accuracy_rate)

accuracy:  0.9


In [16]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Dense(n_hidden, input_dim=n_in))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_hidden))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_hidden))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_out))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.05), metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size)




Epoch 1/1000
8000/8000 [==============================] - 9s 1ms/step - loss: 2.4314 - accuracy: 0.1663
Epoch 2/1000
8000/8000 [==============================] - 3s 393us/step - loss: 2.1988 - accuracy: 0.2059
Epoch 3/1000
8000/8000 [==============================] - 2s 278us/step - loss: 2.0924 - accuracy: 0.2389
Epoch 4/1000
8000/8000 [==============================] - 3s 360us/step - loss: 1.9940 - accuracy: 0.2726
Epoch 5/1000
8000/8000 [==============================] - 3s 356us/step - loss: 2.0451 - accuracy: 0.2697
Epoch 6/1000
8000/8000 [==============================] - 3s 359us/step - loss: 2.0005 - accuracy: 0.2796
Epoch 7/1000
8000/8000 [==============================] - 3s 335us/step - loss: 1.9137 - accuracy: 0.3097
Epoch 8/1000
8000/8000 [==============================] - 3s 333us/step - loss: 1.9489 - accuracy: 0.3002
Epoch 9/1000
8000/8000 [==============================] - 2s 304us/step - loss: 2.0276 - accuracy: 0.2649
Epoch 10/1000
8000/8000 [====================

8000/8000 [==============================] - 3s 363us/step - loss: 1.9596 - accuracy: 0.2931
Epoch 73/1000
8000/8000 [==============================] - 3s 334us/step - loss: 1.9352 - accuracy: 0.3077
Epoch 74/1000
8000/8000 [==============================] - 3s 368us/step - loss: 1.9628 - accuracy: 0.3040
Epoch 75/1000
8000/8000 [==============================] - 4s 517us/step - loss: 1.9448 - accuracy: 0.3022
Epoch 76/1000
8000/8000 [==============================] - 7s 895us/step - loss: 1.9321 - accuracy: 0.3059
Epoch 77/1000
8000/8000 [==============================] - 7s 839us/step - loss: 1.9294 - accuracy: 0.31690s - loss: 1.9287 - 
Epoch 78/1000
8000/8000 [==============================] - 3s 337us/step - loss: 1.9172 - accuracy: 0.3128
Epoch 79/1000
8000/8000 [==============================] - 3s 341us/step - loss: 1.9121 - accuracy: 0.3145
Epoch 80/1000
8000/8000 [==============================] - 3s 362us/step - loss: 1.9260 - accuracy: 0.3055
Epoch 81/1000
8000/8000 [======

8000/8000 [==============================] - 3s 426us/step - loss: 1.9861 - accuracy: 0.2934
Epoch 149/1000
8000/8000 [==============================] - 3s 413us/step - loss: 1.9922 - accuracy: 0.29310s - loss: 1.9
Epoch 150/1000
8000/8000 [==============================] - 3s 429us/step - loss: 1.9866 - accuracy: 0.2875
Epoch 151/1000
8000/8000 [==============================] - 4s 455us/step - loss: 1.9974 - accuracy: 0.2862
Epoch 152/1000
8000/8000 [==============================] - 4s 442us/step - loss: 2.0034 - accuracy: 0.2856
Epoch 153/1000
8000/8000 [==============================] - 3s 424us/step - loss: 2.0087 - accuracy: 0.2760
Epoch 154/1000
8000/8000 [==============================] - 4s 448us/step - loss: 2.0079 - accuracy: 0.2776
Epoch 155/1000
8000/8000 [==============================] - 4s 519us/step - loss: 1.9780 - accuracy: 0.2854
Epoch 156/1000
8000/8000 [==============================] - 8s 938us/step - loss: 1.9939 - accuracy: 0.2848
Epoch 157/1000
8000/8000 [===

8000/8000 [==============================] - 7s 841us/step - loss: 1.9893 - accuracy: 0.2876
Epoch 224/1000
8000/8000 [==============================] - 4s 497us/step - loss: 1.9800 - accuracy: 0.2870
Epoch 225/1000
8000/8000 [==============================] - 3s 349us/step - loss: 1.9707 - accuracy: 0.2991
Epoch 226/1000
8000/8000 [==============================] - 3s 348us/step - loss: 1.9876 - accuracy: 0.2853
Epoch 227/1000
8000/8000 [==============================] - 3s 379us/step - loss: 1.9820 - accuracy: 0.2889
Epoch 228/1000
8000/8000 [==============================] - 3s 351us/step - loss: 1.9777 - accuracy: 0.2911
Epoch 229/1000
8000/8000 [==============================] - 3s 343us/step - loss: 1.9764 - accuracy: 0.2941
Epoch 230/1000
8000/8000 [==============================] - 3s 371us/step - loss: 1.9777 - accuracy: 0.2920
Epoch 231/1000
8000/8000 [==============================] - 3s 381us/step - loss: 1.9925 - accuracy: 0.2905
Epoch 232/1000
8000/8000 [=================

8000/8000 [==============================] - ETA: 0s - loss: 1.9209 - accuracy: 0.30 - 3s 353us/step - loss: 1.9222 - accuracy: 0.3069
Epoch 299/1000
8000/8000 [==============================] - 3s 314us/step - loss: 1.9006 - accuracy: 0.3171
Epoch 300/1000
8000/8000 [==============================] - 3s 334us/step - loss: 1.9156 - accuracy: 0.3137
Epoch 301/1000
8000/8000 [==============================] - 3s 363us/step - loss: 1.9060 - accuracy: 0.3110
Epoch 302/1000
8000/8000 [==============================] - 3s 364us/step - loss: 1.9240 - accuracy: 0.3034
Epoch 303/1000
8000/8000 [==============================] - 3s 375us/step - loss: 1.9502 - accuracy: 0.3035
Epoch 304/1000
8000/8000 [==============================] - 3s 346us/step - loss: 1.9318 - accuracy: 0.3040
Epoch 305/1000
8000/8000 [==============================] - 3s 342us/step - loss: 1.9288 - accuracy: 0.3119
Epoch 306/1000
8000/8000 [==============================] - 3s 364us/step - loss: 1.9216 - accuracy: 0.3103
E

...

8000/8000 [==============================] - 3s 371us/step - loss: 1.8389 - accuracy: 0.3360
Epoch 674/1000
8000/8000 [==============================] - 3s 370us/step - loss: 1.8470 - accuracy: 0.3257
Epoch 675/1000
8000/8000 [==============================] - 3s 343us/step - loss: 1.8392 - accuracy: 0.3331
Epoch 676/1000
8000/8000 [==============================] - 3s 395us/step - loss: 1.8346 - accuracy: 0.3299
Epoch 677/1000
8000/8000 [==============================] - 3s 386us/step - loss: 1.8532 - accuracy: 0.3231
Epoch 678/1000
8000/8000 [==============================] - 3s 372us/step - loss: 1.8524 - accuracy: 0.3162
Epoch 679/1000
8000/8000 [==============================] - 5s 669us/step - loss: 1.8364 - accuracy: 0.3250
Epoch 680/1000
8000/8000 [==============================] - 7s 852us/step - loss: 1.8272 - accuracy: 0.3380
Epoch 681/1000
8000/8000 [==============================] - 6s 749us/step - loss: 1.8468 - accuracy: 0.3281
Epoch 682/1000
8000/8000 [=================

8000/8000 [==============================] - 6s 739us/step - loss: 1.8319 - accuracy: 0.3329
Epoch 749/1000
8000/8000 [==============================] - 7s 840us/step - loss: 1.8445 - accuracy: 0.3284
Epoch 750/1000
8000/8000 [==============================] - 5s 604us/step - loss: 1.8467 - accuracy: 0.3318
Epoch 751/1000
8000/8000 [==============================] - 3s 380us/step - loss: 1.8327 - accuracy: 0.3358
Epoch 752/1000
8000/8000 [==============================] - 3s 348us/step - loss: 1.8573 - accuracy: 0.3275
Epoch 753/1000
8000/8000 [==============================] - 3s 355us/step - loss: 1.8568 - accuracy: 0.3284
Epoch 754/1000
8000/8000 [==============================] - 3s 388us/step - loss: 1.8435 - accuracy: 0.3334
Epoch 755/1000
8000/8000 [==============================] - 3s 382us/step - loss: 1.8296 - accuracy: 0.3321
Epoch 756/1000
8000/8000 [==============================] - 3s 376us/step - loss: 1.8350 - accuracy: 0.3277
Epoch 757/1000
8000/8000 [=================

Epoch 824/1000
8000/8000 [==============================] - 3s 347us/step - loss: 1.8299 - accuracy: 0.3335
Epoch 825/1000
8000/8000 [==============================] - 3s 355us/step - loss: 1.8430 - accuracy: 0.3311
Epoch 826/1000
8000/8000 [==============================] - 3s 361us/step - loss: 1.8365 - accuracy: 0.3324
Epoch 827/1000
8000/8000 [==============================] - 3s 349us/step - loss: 1.8303 - accuracy: 0.3419
Epoch 828/1000
8000/8000 [==============================] - 3s 394us/step - loss: 1.8466 - accuracy: 0.3294
Epoch 829/1000
8000/8000 [==============================] - 3s 401us/step - loss: 1.8369 - accuracy: 0.3302
Epoch 830/1000
8000/8000 [==============================] - 5s 627us/step - loss: 1.8479 - accuracy: 0.3339
Epoch 831/1000
8000/8000 [==============================] - 7s 838us/step - loss: 1.8413 - accuracy: 0.3269
Epoch 832/1000
8000/8000 [==============================] - 6s 808us/step - loss: 1.8450 - accuracy: 0.3296
Epoch 833/1000
8000/8000 [==

8000/8000 [==============================] - 6s 808us/step - loss: 1.8433 - accuracy: 0.3295
Epoch 900/1000
8000/8000 [==============================] - 5s 675us/step - loss: 1.8446 - accuracy: 0.32791s
Epoch 901/1000
8000/8000 [==============================] - 3s 345us/step - loss: 1.8530 - accuracy: 0.3262
Epoch 902/1000
8000/8000 [==============================] - 3s 318us/step - loss: 1.8448 - accuracy: 0.3329
Epoch 903/1000
8000/8000 [==============================] - 3s 336us/step - loss: 1.8223 - accuracy: 0.3406
Epoch 904/1000
8000/8000 [==============================] - 3s 355us/step - loss: 1.8398 - accuracy: 0.3302
Epoch 905/1000
8000/8000 [==============================] - 3s 335us/step - loss: 1.8450 - accuracy: 0.3268
Epoch 906/1000
8000/8000 [==============================] - 3s 326us/step - loss: 1.8439 - accuracy: 0.3293
Epoch 907/1000
8000/8000 [==============================] - 3s 343us/step - loss: 1.8389 - accuracy: 0.3255
Epoch 908/1000
8000/8000 [===============

8000/8000 [==============================] - 3s 353us/step - loss: 1.8329 - accuracy: 0.3282
Epoch 975/1000
8000/8000 [==============================] - 3s 378us/step - loss: 1.8312 - accuracy: 0.3360
Epoch 976/1000
8000/8000 [==============================] - 3s 377us/step - loss: 1.8362 - accuracy: 0.3276
Epoch 977/1000
8000/8000 [==============================] - 3s 359us/step - loss: 1.8439 - accuracy: 0.3340
Epoch 978/1000
8000/8000 [==============================] - 3s 371us/step - loss: 1.8479 - accuracy: 0.3324
Epoch 979/1000
8000/8000 [==============================] - 3s 397us/step - loss: 1.8572 - accuracy: 0.3324
Epoch 980/1000
8000/8000 [==============================] - 3s 376us/step - loss: 1.8349 - accuracy: 0.3268
Epoch 981/1000
8000/8000 [==============================] - 4s 548us/step - loss: 1.8432 - accuracy: 0.3271
Epoch 982/1000
8000/8000 [==============================] - 7s 840us/step - loss: 1.8353 - accuracy: 0.3316
Epoch 983/1000
8000/8000 [=================

In [17]:
loss_and_accuracy = model.evaluate(X_test, Y_test)
print(loss_and_accuracy)

2000/2000 [==============================] - 0s 130us/step
[1.4303041610717773, 0.445499986410141]


In [18]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Dense(n_hidden, input_dim=n_in))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_hidden))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_hidden))
model.add(Activation('tanh'))
model.add(Dropout(0.5))

model.add(Dense(n_out))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.001), metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50, batch_size=batch_size)

Epoch 1/50
8000/8000 [==============================] - 4s 462us/step - loss: 2.6536 - accuracy: 0.1410
Epoch 2/50
8000/8000 [==============================] - 3s 379us/step - loss: 2.3083 - accuracy: 0.2257
Epoch 3/50
8000/8000 [==============================] - 6s 765us/step - loss: 2.0523 - accuracy: 0.3014
Epoch 4/50
8000/8000 [==============================] - 7s 845us/step - loss: 1.8778 - accuracy: 0.3509
Epoch 5/50
8000/8000 [==============================] - 6s 723us/step - loss: 1.7022 - accuracy: 0.4031
Epoch 6/50
8000/8000 [==============================] - 3s 379us/step - loss: 1.5712 - accuracy: 0.4470
Epoch 7/50
8000/8000 [==============================] - 3s 427us/step - loss: 1.4631 - accuracy: 0.4949
Epoch 8/50
8000/8000 [==============================] - 3s 367us/step - loss: 1.3640 - accuracy: 0.5270
Epoch 9/50
8000/8000 [==============================] - 3s 336us/step - loss: 1.2905 - accuracy: 0.5487
Epoch 10/50
8000/8000 [==============================] - 3s 371u

In [19]:
loss_and_accuracy = model.evaluate(X_test, Y_test)
print(loss_and_accuracy)

2000/2000 [==============================] - 1s 320us/step
[0.42715695732831954, 0.8784999847412109]


In [20]:
def inference(x, keep_prob, n_in, n_hiddens, n_out):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)
    
    def bias_variable(shape):
        initial = tf.zeros(shape)
        return tf.Variable(initial)
    
    # 입력층 - 은닉층, 은닉층 - 은닉층
    for i, n_hidden in enumerate(n_hiddens):
        if i == 0:
            input = x
            input_dim = n_in
        else:
            input = output
            input_dim = n_hiddens[i-1]
            
        W = weight_variable([input_dim, n_hidden])
        b = bias_variable([n_hidden])
        
        h = tf.nn.relu(tf.matmul(input, W) + b)
        output = tf.nn.dropout(h, keep_prob)
        
    # 은닉층 - 출력층
    W_out = weight_variable([n_hidden[-1], n_out])
    b_out = bias_variable([n_out])
    y = tf.nn.softmax(tf.matmul(output, W_out) + b_out)
    return y

def loss(y, t):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y), reduction_indices=[1]))
    return cross_entropy

def training(loss):
    optimizer = tf.train.GradientDescentOptimizer(0.001)
    train_step = optimizer.minimize(loss)
    return train_step